In [1]:
from langgraph.graph import END, START, StateGraph
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import requests
import random
import time

In [2]:
llm = ChatOllama(model="llama3.2:3b")

In [3]:
#Tools
@tool
def duckduckgo_search(query: str):
    """Search anything using DuckDuckGo Search Engine."""
    search = DuckDuckGoSearchRun()
    return search.run(query)

In [4]:
@tool
def arithmetic(operation: str):
    """Perform basic arithmetic operations.
    Format: 'add 2 3', 'sub 10 4', 'mul 3 5', 'div 8 2'
    """
    parts = operation.split()
    if len(parts) != 3:
        return "Format error: use 'add 2 3'"

    op, a, b = parts[0], float(parts[1]), float(parts[2])
    
    if op == "add":
        return a + b
    elif op == "sub":
        return a - b
    elif op == "mul":
        return a * b
    elif op == "div":
        return a / b if b != 0 else "Error: cannot divide by zero"
    else:
        return "Unknown operation. Use add/sub/mul/div."


In [5]:
@tool
def weather_api(location: str):
    """Get current weather for a city using free Open-Meteo API."""
    try:
        # 1. geocoding to get latitude & longitude
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo = requests.get(geo_url, params={"name": location, "count": 1}).json()

        if "results" not in geo:
            return "Location not found."

        lat = geo["results"][0]["latitude"]
        lon = geo["results"][0]["longitude"]

        # 2. get current weather
        weather_url = "https://api.open-meteo.com/v1/forecast"
        weather = requests.get(
            weather_url,
            params={
                "latitude": lat,
                "longitude": lon,
                "current_weather": True
            }
        ).json()

        current = weather.get("current_weather", {})

        return {
            "location": location,
            "temperature_c": current.get("temperature"),
            "windspeed_kmh": current.get("windspeed"),
            "weather_code": current.get("weathercode"),
        }

    except Exception as e:
        return f"Error: {str(e)}"


In [6]:
@tool
def stock_price(symbol: str):
    """Get real-time stock price using free Yahoo Finance API."""
    try:
        url = f"https://query1.finance.yahoo.com/v7/finance/quote?symbols={symbol}"
        data = requests.get(url).json()

        if "quoteResponse" not in data or len(data["quoteResponse"]["result"]) == 0:
            return "Stock symbol not found."

        result = data["quoteResponse"]["result"][0]

        return {
            "symbol": symbol,
            "price": result.get("regularMarketPrice"),
            "change": result.get("regularMarketChange"),
            "percent_change": result.get("regularMarketChangePercent"),
            "market_cap": result.get("marketCap"),
            "currency": result.get("currency")
        }
    except Exception as e:
        return f"Error: {str(e)}"


In [7]:
@tool
def wikipedia_search(query: str):
    """Search Wikipedia and return a summary."""
    try:
        # 1. Search
        search_url = "https://en.wikipedia.org/w/api.php"
        search_params = {
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json"
        }
        search_result = requests.get(search_url, params=search_params).json()

        if "query" not in search_result or len(search_result["query"]["search"]) == 0:
            return "No results found on Wikipedia."

        page_title = search_result["query"]["search"][0]["title"]

        # 2. Get summary
        summary_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title}"
        summary_data = requests.get(summary_url).json()

        return {
            "title": summary_data.get("title"),
            "summary": summary_data.get("extract"),
            "url": summary_data.get("content_urls", {}).get("desktop", {}).get("page")
        }

    except Exception as e:
        return f"Error: {str(e)}"


In [8]:
tools = [
    duckduckgo_search,
    arithmetic,
    weather_api,
    stock_price,
    wikipedia_search
]

llm_with_tools = llm.bind(tools=tools)


In [9]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
def chat_node(state: ChatState):
    

In [ ]:
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_node("tools",tools_node)
